In [19]:
from openai import AzureOpenAI
from pydantic import BaseModel
from neopipe.result import Ok, Result, Err, ExecutionResult
from neopipe.task import FunctionSyncTask, ClassSyncTask
from neopipe.pipeline import SyncPipeline
from dotenv import load_dotenv, find_dotenv
import os
import logging

In [2]:
logging.getLogger("openai").setLevel(logging.ERROR)
logging.getLogger("httpx").setLevel(logging.ERROR)
# logging.getLogger("neopipe").setLevel(logging.ERROR)

In [4]:
load_dotenv(find_dotenv("../.env"), override=True)

True

In [5]:
LLM_CLIENT = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
)

In [6]:
# ───────────────────────────────────────────────────────────────
# 1) In‑memory “database”
# ───────────────────────────────────────────────────────────────
DB = [
    {"id": 1, "text": "The quick brown fox jumps over the lazy dog"},
    {"id": 2, "text": "A journey of a thousand miles begins with a single step"},
    {"id": 3, "text": "To be or not to be, that is the question"},
]


In [7]:
class SearchResult(BaseModel):
    query: str
    hits: list[dict]

In [8]:
@FunctionSyncTask.decorator()
def search_task(res: Result[str, str]) -> Result[SearchResult, str]:
    """
    Given a query string in res.unwrap(), returns SearchResult:
    """
    if res.is_err():
        return res

    query = res.unwrap()
    hits = [record for record in DB if query.lower() in record["text"].lower()]
    return Ok(SearchResult(query=query, hits=hits))

In [9]:
search_result = search_task(Result.Ok("quick"))

2025-05-08 15:02:24 - neopipe.task - INFO - [search_task] Attempt 1 - Task ID: 12139c28-fc12-407f-a2d9-94e3fcca39d9
2025-05-08 15:02:24 - neopipe.task - INFO - [search_task] Success on attempt 1


In [9]:
# search_result.unwrap().model_dump()

In [10]:
@FunctionSyncTask.decorator()
def build_context_task(res: Result[SearchResult, str]) -> Result[list[dict], str]:
    """
    Given the context dict {query,hits}, build the OpenAI chat messages:
      [ system, user, assistant(context-of-hits) ]
    """
    if res.is_err():
        return res

    ctx = res.unwrap().model_dump()
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that summarizes search results."
        },
        {
            "role": "user",
            "content": f"I’m looking for records containing: “{ctx['query']}”"
        },
        {
            "role": "assistant",
            "content": (
                f"Found {len(ctx['hits'])} record(s):\n" +
                "\n".join(f"- {h['text']}" for h in ctx["hits"])
            )
        },
    ]
    return Ok(messages)

In [11]:
build_context_task(search_result)

2025-05-08 15:02:31 - neopipe.task - INFO - [build_context_task] Attempt 1 - Task ID: cbfafa37-2a55-49ce-bd67-3f87c2ffec9a
2025-05-08 15:02:31 - neopipe.task - INFO - [build_context_task] Success on attempt 1


Ok([{'role': 'system', 'content': 'You are a helpful assistant that summarizes search results.'}, {'role': 'user', 'content': 'I’m looking for records containing: “quick”'}, {'role': 'assistant', 'content': 'Found 1 record(s):\n- The quick brown fox jumps over the lazy dog'}])

In [12]:
class OpenAITask(ClassSyncTask[list[dict], str]):
    """
    Sends the assembled messages list to OpenAI and returns the assistant’s reply.
    """
    def __init__(self, client: AzureOpenAI, model: str ="gpt-4o"):
        super().__init__()
        self.client = client
        self.model = model

    def execute(self, res: Result[list[dict], str]) -> Result[str, str]:
        if res.is_err():
            return res
        messages = res.unwrap()
        try:
            resp = self.client.chat.completions.create(
                model=self.model,
                messages=messages
            )
            return Ok(resp.choices[0].message.content.strip())
        except Exception as e:
            return Err(str(e))

In [13]:
pipeline = SyncPipeline.from_tasks(
    [search_task, build_context_task, OpenAITask(client=LLM_CLIENT)],
    name="SearchAndRespond"
)

In [ ]:
def run_search_and_respond(user_query: str):
    # Wrap the user query in Ok(...)
    result = pipeline.run(Ok(user_query), debug=True)
    return result

In [16]:
q = "quick"
response = run_search_and_respond(q)

2025-05-08 15:03:45 - neopipe.task - INFO - [search_task] Attempt 1 - Task ID: 12139c28-fc12-407f-a2d9-94e3fcca39d9
2025-05-08 15:03:45 - neopipe.task - INFO - [search_task] Success on attempt 1
2025-05-08 15:03:45 - neopipe.task - INFO - [build_context_task] Attempt 1 - Task ID: cbfafa37-2a55-49ce-bd67-3f87c2ffec9a
2025-05-08 15:03:45 - neopipe.task - INFO - [build_context_task] Success on attempt 1
2025-05-08 15:03:45 - neopipe.task - INFO - [OpenAITask] Attempt 1 - Task ID: 53a4f59e-d2f9-4f70-a6d2-d45d52d68c13
2025-05-08 15:03:46 - neopipe.task - INFO - [OpenAITask] Success on attempt 1


In [17]:
print(response)

ExecutionResult(result=Ok('Sure, here\'s one record containing "quick":\n\n- "The quick brown fox jumps over the lazy dog."'), execution_time=0.836s, trace=Trace(steps=[('SearchAndRespond', Ok('quick')), ('search_task', Ok(SearchResult(query='quick', hits=[{'id': 1, 'text': 'The quick brown fox jumps over the lazy dog'}]))), ('build_context_task', Ok([{'role': 'system', 'content': 'You are a helpful assistant that summarizes search results.'}, {'role': 'user', 'content': 'I’m looking for records containing: “quick”'}, {'role': 'assistant', 'content': 'Found 1 record(s):\n- The quick brown fox jumps over the lazy dog'}])), ('OpenAITask', Ok('Sure, here\'s one record containing "quick":\n\n- "The quick brown fox jumps over the lazy dog."'))]))


In [22]:
def result_nicely(res: ExecutionResult, query: str) -> None:
    """
    Print the result nicely.
    """
    if res.result.is_err():
        print("❌ Pipeline error:", res.result.err())
    else:
        final = res.result.unwrap()
        trace = res.trace
        print("🤖 AI Response:\n", final, "\n")
        print("—-- Trace —--")
        print(f"Input query: {query!r}")
        for task_name, step_res in trace.steps:
            print(f"{task_name:20} → {step_res}")

In [24]:
result_nicely(response, q)

🤖 AI Response:
 Sure, here's one record containing "quick":

- "The quick brown fox jumps over the lazy dog." 

—-- Trace —--
Input query: 'quick'
SearchAndRespond     → Ok('quick')
search_task          → Ok(SearchResult(query='quick', hits=[{'id': 1, 'text': 'The quick brown fox jumps over the lazy dog'}]))
build_context_task   → Ok([{'role': 'system', 'content': 'You are a helpful assistant that summarizes search results.'}, {'role': 'user', 'content': 'I’m looking for records containing: “quick”'}, {'role': 'assistant', 'content': 'Found 1 record(s):\n- The quick brown fox jumps over the lazy dog'}])
OpenAITask           → Ok('Sure, here\'s one record containing "quick":\n\n- "The quick brown fox jumps over the lazy dog."')
